# 運用系処理

## ライブラリインポート

In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from elastic_manager.elastic_manager import ElasticManager
import pandas as pd
pd.set_option('display.max_rows', 1000) # 最大表示数

## インデックス一覧
インデックスの名前とドキュメント数、サイズ（kB）を一覧で表示します。  
indexを指定しない場合、すべてのインデックスが表示されます。  
indexを指定した場合、そのパターンに一致するインデックス一覧が表示されます。  
例）ElasticManager.show_indices(index="rawdata-*")  
⇒ rawdataインデックスの一覧が表示されます。

In [7]:
# 例）ElasticManager.show_indices(index="rawdata-*")
ElasticManager.show_indices(index="shots-*-data")

,index,docs.count,store.size
0,shots-20201201010000-data,9281299,1199088
1,shots-20210129114815-data,0,0


## インデックス削除
DELETE_INDEXに削除するインデックス名を入力します。ワイルドカード(\*)が使用可能です。  
例）rawdata-20201201\*  
⇒ 2020/12/01のrawdataデータインデックスがすべて削除されます。

In [8]:
DELETE_INDEX = 'shots-20210129114815-data'

In [9]:
ElasticManager.delete_index(DELETE_INDEX)

## データ削除（連番指定）
連番指定でデータを削除します。以下のパラメータを指定してください。

* TARGET_INDEX：データ削除対象のインデックス名  
* START_SEQ_NUM：開始番号  
* END_SEQ_NUM：終了番号

In [8]:
TARGET_INDEX = 'rawdata-20210118153002'
START_SEQ_NUM = 0
END_SEQ_NUM = 10

In [9]:
ElasticManager.delete_data_by_seq_num(index=TARGET_INDEX, start=START_SEQ_NUM, end=END_SEQ_NUM)

## データ削除（ショット番号指定）
ショット番号指定でデータを削除します。以下のパラメータを指定してください。  

* TARGET_INDEX：データ削除対象のインデックス名  
* SHOT_NUMBER：ショット番号

In [5]:
TARGET_INDEX = 'shots-20201201010000-data'
SHOT_NUMBER = 2

In [6]:
ElasticManager.delete_data_by_shot_num(index=TARGET_INDEX, shot_number=SHOT_NUMBER)